## Multi-layer perceptron

I am not even going to try and write a better intro. to neural nets than this...

https://jalammar.github.io/visual-interactive-guide-basics-neural-networks/

### Softmax Equation

Given an array of values of length n, the softmax of value i in the array is:

$$\frac{e^{i}}{\sum_{j}^{n}e^{j}}$$

In [6]:
import numpy as np

values = np.array([1.0, 3.0, 8.0, 4.0, 12.0])
exp_values = np.exp(values)
softmax = exp_values / sum(exp_values)
print([round(x,2) for x in softmax])
print(sum(softmax))

[0.0, 0.0, 0.02, 0.0, 0.98]
1.0


## Example using Python

In [66]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix
from __future__ import division

In [43]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

def vectorize_image(images):
    scaled_images = images / 255
    return images.reshape(scaled_images.shape[0],-1)

x_train = vectorize_image(x_train)
x_test = vectorize_image(x_test)

In [44]:
model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

In [64]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy')

In [65]:
model.fit(x_train, y_train, nb_epoch=5, batch_size=32, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 8s - loss: 5.5664 - val_loss: 5.6349
Epoch 2/5
48000/48000 [==============================] - 8s - loss: 5.5635 - val_loss: 5.5425
Epoch 3/5
48000/48000 [==============================] - 8s - loss: 5.4864 - val_loss: 5.4520
Epoch 4/5
48000/48000 [==============================] - 9s - loss: 5.4804 - val_loss: 5.4800
Epoch 5/5
48000/48000 [==============================] - 9s - loss: 5.4620 - val_loss: 5.4380


In [81]:
test_predictions = np.argmax(model.predict(x_test),1)
y_test_sparse = np.argmax(y_test, 1)

In [82]:
confusion_matrix(y_test_sparse, test_predictions)

array([[   0,    0,  360,    0,    0,  297,  279,   20,   19,    5],
       [   0, 1116,    5,    0,    0,    3,    6,    1,    4,    0],
       [   0,   10,  967,    0,    0,    7,   11,   14,   15,    8],
       [   0,   12,  192,    0,    0,  472,   21,   32,  260,   21],
       [   0,    7,   15,    0,    0,    0,   34,    4,   27,  895],
       [   0,    1,   17,    0,    0,  810,   30,    5,   19,   10],
       [   0,    3,    5,    0,    0,    4,  937,    1,    4,    4],
       [   0,   10,   16,    0,    0,    0,    1,  985,    1,   15],
       [   0,    3,   10,    0,    0,   10,   16,   12,  905,   18],
       [   0,   13,    1,    0,    0,   15,    1,   21,   15,  943]])

In [83]:
np.sum(y_test_sparse == test_predictions) / test_predictions.shape

array([ 0.6663])